In [ ]:
import torch
import detectron2

print("CUDA Available:", torch.cuda.is_available())  # Should print True
print("GPU Device Name:", torch.cuda.get_device_name(0)) 

In [ ]:
from detectron2.data import MetadataCatalog, DatasetCatalog

# Unregister datasets if already registered
for d in ["btxrd_dataset_train", "btxrd_dataset_val"]:
    if d in DatasetCatalog:
        DatasetCatalog.remove(d)
    if d in MetadataCatalog:
        MetadataCatalog.remove(d)

In [ ]:
from detectron2.data.datasets import register_coco_instances

register_coco_instances("btxrd_dataset_train", {}, "D:/BTXRD-Dataset/BTXRD-Yolo/train/labels/coco_annotations.json", "D:/BTXRD-Dataset/BTXRD-Yolo/train/images")
register_coco_instances("btxrd_dataset_val", {}, "D:/BTXRD-Dataset/BTXRD-Yolo/val/labels/coco_annotations.json", "D:/BTXRD-Dataset/BTXRD-Yolo/val/images")

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader, build_detection_train_loader
from detectron2.data import detection_utils as utils
from detectron2.data import transforms as T
from detectron2.data.datasets import register_coco_instances
from detectron2.data.dataset_mapper import DatasetMapper
import os
import torch
import logging

class MyTrainer(DefaultTrainer):
    def __init__(self, cfg):
        super().__init__(cfg)
        self.best_ap = 0
        self.best_iter = 0

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR,"inference")
        return COCOEvaluator(dataset_name, cfg, True, output_folder)

    @classmethod
    def build_train_loader(cls, cfg):
        # Define your augmentations here
        augmentation = [
            T.RandomFlip(prob=0.5, horizontal=True, vertical=False),
            T.RandomCrop("relative_range", (0.8, 0.8)),
            T.RandomRotation(angle=[0, 90, 180, 270]),
            T.ResizeShortestEdge(short_edge_length=(640, 800), max_size=1333, sample_style='choice'),
            T.RandomBrightness(0.8, 1.2),
            T.RandomContrast(0.8, 1.2),
            T.RandomSaturation(0.8, 1.2),
            T.RandomLighting(0.2),
        ]

        return build_detection_train_loader(
            cfg,
            mapper=DatasetMapper(cfg, is_train=True, augmentations=augmentation),
            )


In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg, CfgNode
from detectron2 import model_zoo
from detectron2.engine import hooks
import os

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("btxrd_dataset_train",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.OUTPUT_DIR = "./detectron_runs" 
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_best_6208.pth")
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00025 
cfg.SOLVER.MAX_ITER = 1000 
# cfg.SOLVER.STEPS = (40000, 50000) 
# cfg.SOLVER.GAMMA = 0.1  
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2 
cfg.DATASETS.TEST = ("btxrd_dataset_val",)
cfg.TEST.EVAL_PERIOD = 100
cfg.SOLVER.BEST_CHECKPOINTER = CfgNode({"ENABLED": True})  
cfg.SOLVER.BEST_CHECKPOINTER.METRIC = "bbox/AP50"  
cfg.SOLVER.BEST_CHECKPOINTER.MODE = "max"
cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = False
cfg.DATALOADER.SAMPLER_TRAIN = "RepeatFactorTrainingSampler"
cfg.DATALOADER.REPEAT_THRESHOLD = 0.05 

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = MyTrainer(cfg)

trainer.register_hooks([
  hooks.BestCheckpointer(
    cfg.TEST.EVAL_PERIOD,  
    trainer.checkpointer,
    cfg.SOLVER.BEST_CHECKPOINTER.METRIC,
    mode=cfg.SOLVER.BEST_CHECKPOINTER.MODE
  )
])

trainer.resume_or_load(resume=False)
trainer.train()


In [ ]:
# Evaluate the model on the validation set

from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.engine import DefaultPredictor
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2 import model_zoo

# Load config
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.OUTPUT_DIR = "./detectron_runs" 
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_best_6280.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.01
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2 
cfg.DATASETS.TEST = ("btxrd_dataset_val", )


model = DefaultTrainer.build_model(cfg)
DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)

# Run evaluation
evaluator = COCOEvaluator("btxrd_dataset_val", cfg, False, output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "btxrd_dataset_val")
inference_on_dataset(model, val_loader, evaluator)